# Explore here

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
import math
import utils as eda
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from pickle import dump

In [7]:
url =https://raw.githubusercontent.com/4GeeksAcademy/decision-tree-project-tutorial/main/diabetes.csv
total_data = pd.read_csv(url)  

total_data.head()

SyntaxError: invalid decimal literal (4235533426.py, line 1)